<a href="https://colab.research.google.com/github/phnuness/emd-desafio-junior-data-scientist/blob/main/analise_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install basedosdados --use-deprecated=legacy-resolver


In [ ]:
# Importando a biblioteca
import basedosdados as bd
import pandas as pd

In [ ]:
# Realizando a query na tabela de chamados para criar o dataframe
query = '''

SELECT
  id_chamado, data_inicio, data_fim, id_bairro, tipo, subtipo
FROM
  `datario.administracao_servicos_publicos.chamado_1746`

'''
df_chamados = bd.read_sql(query, billing_project_id="cellular-tide-414417")

Downloading: 100%|██████████| 10742832/10742832 [16:32<00:00, 10823.75rows/s]


In [ ]:
df_chamados.head()

In [ ]:
df_chamados.info()

1. Quantos chamados foram abertos no dia 01/04/2023?

In [ ]:
# Filtrando a quantidade de chamados abertos na data 01/04/2023
len(df_chamados.loc[(df_chamados.data_inicio >= '2023-04-01') & (df_chamados.data_inicio < '2023-04-02')])

73

2. Qual o tipo de chamado que teve mais reclamações no dia 01/04/2023?

In [ ]:
# Filtrando a coluna tipo para identificar o tipo com mais reclamações no dia 01/04/2023
df_chamados['tipo'].loc[(df_chamados.data_inicio >= '2023-04-01') & (df_chamados.data_inicio < '2023-04-02')].value_counts().head(1)


Poluição sonora    24
Name: tipo, dtype: int64

3. Quais os nomes dos 3 bairros que mais tiveram chamados abertos nesse dia?

In [ ]:
# Realizando a query na tabela de bairros para criar o dataframe
query = '''

SELECT
   id_bairro, subprefeitura, nome as nome_bairro
FROM
  `datario.dados_mestres.bairro`

'''
df_bairros = bd.read_sql(query, billing_project_id="cellular-tide-414417")

Downloading: 100%|██████████| 164/164 [00:00<00:00, 857.05rows/s]


In [ ]:
# Criando um cada segundo dataframe com o join entre as tabelas chamado e bairro
df_chamados_bairros = df_chamados[['id_chamado','data_inicio','id_bairro']].merge(df_bairros[['id_bairro', 'nome_bairro', 'subprefeitura']],
                                                                  how='left', on='id_bairro')

In [ ]:
# Filtrando a coluna nome_bairro para identificar o bairro com mais reclamações no dia 01/04/2023
df_chamados_bairros['nome_bairro'].loc[(df_chamados_bairros.data_inicio >= '2023-04-01') &
 (df_chamados_bairros.data_inicio < '2023-04-02')].value_counts().head(3)


Engenho de Dentro    8
Leblon               6
Campo Grande         6
Name: nome_bairro, dtype: int64

4. Qual o nome da subprefeitura com mais chamados abertos nesse dia?

In [ ]:
# Filtrando a coluna subprefeitura para identificar a subprefeitura com mais reclamações no dia 01/04/2023
df_chamados_bairros['subprefeitura'].loc[(df_chamados_bairros.data_inicio >= '2023-04-01') &
 (df_chamados_bairros.data_inicio < '2023-04-02')].value_counts().head(1)

Zona Norte    25
Name: subprefeitura, dtype: int64

5. Existe algum chamado aberto nesse dia que não foi associado a um bairro ou   subprefeitura na tabela de bairros? Se sim, por que isso acontece?

  R: Existe um chamado e acontece por motivos de dados ausentes ou não informados (null)

In [ ]:
# Filtrando o df2 para identificar a se existe algum chamado aberto sem bairro ou subprefeitura definido
df_chamados_bairros.loc[(df_chamados_bairros.data_inicio >= '2023-04-01') &
 (df_chamados_bairros.data_inicio < '2023-04-02') & (df_chamados_bairros.id_bairro.isnull())]

,id_chamado,data_inicio,id_bairro,nome_bairro,subprefeitura
3005757,18516246,2023-04-01 00:55:38,None,NaN,NaN


6. Quantos chamados com o subtipo "Perturbação do sossego" foram abertos desde 01/01/2022 até 31/12/2023 (incluindo extremidades)?

  R:

In [ ]:
# Filtrando o df para identificar a quantidade de chamados com o subtipo 'Perturbação do sossego'
df_chamados['subtipo'].loc[(df_chamados.data_inicio >= '2022-01-01') &
 (df_chamados.data_inicio <= '2023-12-31') & (df_chamados.subtipo == 'Perturbação do sossego')].count()

42408

7. Selecione os chamados com esse subtipo que foram abertos durante os eventos contidos na tabela de eventos (Reveillon, Carnaval e Rock in Rio).

In [ ]:
# Realizando a query na tabela de eventos para criar o dataframe
query = '''

SELECT
  *
FROM
  `datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos`

'''
df_eventos = bd.read_sql(query, billing_project_id="cellular-tide-414417")

Downloading: 100%|██████████| 4/4 [00:00<00:00, 18.10rows/s]


In [ ]:
# Convertando as colunas data_inicial e data_final do df de eventos para datetime
df_eventos['data_inicial'] = pd.to_datetime(df_eventos['data_inicial'])
df_eventos['data_final'] = pd.to_datetime(df_eventos['data_final'])

In [ ]:
# Selecionando os chamados com subtipo "Perturbação do sossego"
df_chamados_perturbacao = df[df['subtipo'] == 'Perturbação do sossego']

In [ ]:
# Função que verifica se a data de início do chamado está dentro do intervalo de algum dos eventos
def get_evento(data_inicio):
    eventos = df_eventos[(data_inicio >= df_eventos['data_inicial']) & (data_inicio <= df_eventos['data_final'])]
    if len(eventos) > 0:
        return eventos.iloc[0]['evento']
    else:
        return None

# Criando a nova coluna "evento" no dataframe df_chamados
df_chamados_perturbacao['evento'] = df_chamados_perturbacao['data_inicio'].apply(get_evento)

# Apresentando a seleção de chamados
df_chamados_perturbacao = df_chamados_perturbacao.loc[~df_chamados_perturbacao.evento.isna()]
df_chamados_perturbacao

8. Quantos chamados desse subtipo foram abertos em cada evento?

In [ ]:
# Realizando um groupby por eventos no df filtrado com chamados do subtipo "Perturbação do sossego" df_chamados_perturbacao
quantidade_chamados_evento = df_chamados_perturbacao.groupby(['evento'])['id_chamado'].count()
quantidade_chamados_evento

evento
Carnaval       197
Reveillon       79
Rock in Rio    518
Name: id_chamado, dtype: int64

9. Qual evento teve a maior média diária de chamados abertos desse subtipo?

In [ ]:
# Calculando a duração de cada chamado
duracao_chamado = df_chamados_perturbacao['data_fim'] - df_chamados_perturbacao['data_inicio']

# Calculando a média diária de chamados por evento
media_diaria = duracao_chamado.groupby(df_chamados_perturbacao['evento']).mean()

# Obtendo o evento com a maior média diária
evento_maior_media = media_diaria.idxmax()

print("O evento com a maior média diária de chamados abertos desse subtipo é:", evento_maior_media)

O evento com a maior média diária de chamados abertos desse subtipo é: Rock in Rio


In [ ]:
df_chamados_perturbacao2 = df_chamados_perturbacao
df_chamados_perturbacao2['data_formatada'] = df_chamados_perturbacao2['data_inicio'].dt.strftime('%d/%m/%Y')

10. Compare as médias diárias de chamados abertos desse subtipo durante os eventos específicos (Reveillon, Carnaval e Rock in Rio) e a média diária de chamados abertos desse subtipo considerando todo o período de 01/01/2022 até 31/12/2023.